# Spatial Joins Exercises

Here\'s a reminder of some of the functions we have seen. Hint: they
should be useful for the exercises!

-   `sum(expression)`: aggregate to
    return a sum for a set of records
-   `count(expression)`: aggregate to
    return the size of a set of records
-   `ST_Area(geometry)` returns the
    area of the polygons
-   `ST_AsText(geometry)` returns WKT `text`
-   `ST_Contains(geometry A, geometry B)` returns the true if geometry A contains geometry B
-   `ST_Distance(geometry A, geometry B)` returns the minimum distance between geometry A and
    geometry B
-   `ST_DWithin(geometry A, geometry B, radius)` returns the true if geometry A is radius distance or less from geometry B
-   `ST_GeomFromText(text)` returns `geometry`
-   `ST_Intersects(geometry A, geometry B)` returns the true if geometry A intersects geometry B
-   `ST_Length(linestring)` returns the length of the linestring
-   `ST_Touches(geometry A, geometry B)` returns the true if the boundary of geometry A touches geometry B
-   `ST_Within(geometry A, geometry B)` returns the true if geometry A is within geometry B


Uncomment and run the following cell to install the required packages.


In [22]:
%pip install duckdb leafmap lonboard -q
import duckdb
import leafmap

Download the [nyc_data.zip](https://github.com/opengeos/data/raw/main/duckdb/nyc_data.zip) dataset using leafmap. The zip file contains the following datasets. Create a new DuckDB database and import the datasets into the database. Each dataset should be imported into a separate table.

- nyc_census_blocks
- nyc_homicides
- nyc_neighborhoods
- nyc_streets
- nyc_subway_stations

In [23]:
url = "https://github.com/opengeos/data/raw/main/duckdb/nyc_data.zip"
leafmap.download_file(url, unzip=True)

nyc_data.zip already exists. Skip downloading. Set overwrite=True to overwrite.


'/content/nyc_data.zip'

In [24]:
con = duckdb.connect()

con.execute("INSTALL spatial; LOAD spatial;")
# con.install_extension("spatial")
# con.load_extension("spatial")

In [25]:
datasets = [
    "nyc_census_blocks",
    "nyc_homicides",
    "nyc_neighborhoods",
    "nyc_streets",
    "nyc_subway_stations"
]

In [26]:
for table in datasets:
    # ST_Read automatically pulls data from the associated .dbf and .prj files
    con.execute(f"CREATE OR REPLACE TABLE {table} AS SELECT * FROM ST_Read('{table}.shp');")
    print(f"Finished importing: {table}")

Finished importing: nyc_census_blocks
Finished importing: nyc_homicides
Finished importing: nyc_neighborhoods
Finished importing: nyc_streets
Finished importing: nyc_subway_stations


In [27]:
print(con.execute("SHOW TABLES;").df())

                  name
0    nyc_census_blocks
1        nyc_homicides
2    nyc_neighborhoods
3          nyc_streets
4  nyc_subway_stations


In [36]:
con.execute("SELECT * FROM nyc_census_blocks LIMIT 5").df()

,BLKID,POPN_TOTAL,POPN_WHITE,POPN_BLACK,POPN_NATIV,POPN_ASIAN,POPN_OTHER,BORONAME,geom
0,360850009001000,97,51,32,1,5,8,Staten Island,"[2, 4, 0, 0, 0, 0, 0, 0, 55, 3, 13, 73, 151, 8..."
1,360850020011000,66,52,2,0,7,5,Staten Island,"[2, 4, 0, 0, 0, 0, 0, 0, 178, 58, 13, 73, 72, ..."
2,360850040001000,62,14,18,2,25,3,Staten Island,"[2, 4, 0, 0, 0, 0, 0, 0, 82, 227, 12, 73, 55, ..."
3,360850074001000,137,92,12,0,13,20,Staten Island,"[2, 4, 0, 0, 0, 0, 0, 0, 204, 85, 13, 73, 103,..."
4,360850096011000,289,230,0,0,32,27,Staten Island,"[2, 4, 0, 0, 0, 0, 0, 0, 107, 247, 12, 73, 7, ..."


1. **What subway station is in \'Little Italy\'? What subway route is it on?**

In [30]:
con.sql("""
SELECT
    s.name AS station_name,
    s.routes
FROM
    nyc_subway_stations AS s,
    nyc_neighborhoods AS n
WHERE
    n.name = 'Little Italy'
    AND ST_Intersects(s.geom, n.geom)
""")

┌──────────────┬─────────┐
│ station_name │ ROUTES  │
│   varchar    │ varchar │
├──────────────┼─────────┤
│ Spring St    │ 6       │
└──────────────┴─────────┘

2. **What are all the neighborhoods served by the 6-train?** (Hint: The `routes` column in the `nyc_subway_stations` table has values like \'B,D,6,V\' and \'C,6\')


In [38]:
con.sql("""
SELECT DISTINCT n.name
FROM nyc_neighborhoods AS n, nyc_subway_stations AS s
WHERE
s.routes LIKE '%,6%' AND ST_Contains(n.geom, s.geom)
""")

┌────────────────────┐
│        NAME        │
│      varchar       │
├────────────────────┤
│ Murray Hill        │
│ Upper East Side    │
│ Financial District │
│ East Harlem        │
│ Greenwich Village  │
│ Midtown            │
└────────────────────┘

In [40]:
## correct answer
con.sql("""
SELECT DISTINCT n.name, n.boroname
FROM nyc_subway_stations AS s
JOIN nyc_neighborhoods AS n
ON ST_Contains(n.geom, s.geom)
WHERE strpos(s.routes,'6') > 0;
""")

┌────────────────────┬───────────┐
│        NAME        │ BORONAME  │
│      varchar       │  varchar  │
├────────────────────┼───────────┤
│ Midtown            │ Manhattan │
│ Upper East Side    │ Manhattan │
│ Hunts Point        │ The Bronx │
│ East Harlem        │ Manhattan │
│ South Bronx        │ The Bronx │
│ Yorkville          │ Manhattan │
│ Gramercy           │ Manhattan │
│ Mott Haven         │ The Bronx │
│ Murray Hill        │ Manhattan │
│ Soundview          │ The Bronx │
│ Greenwich Village  │ Manhattan │
│ Financial District │ Manhattan │
│ Parkchester        │ The Bronx │
│ Little Italy       │ Manhattan │
│ Chinatown          │ Manhattan │
├────────────────────┴───────────┤
│ 15 rows              2 columns │
└────────────────────────────────┘

3. **After 9/11, the \'Battery Park\' neighborhood was off limits for several days. How many people had to be evacuated?**

In [37]:
query = """
SELECT SUM(c.POPN_TOTAL) AS population
FROM nyc_census_blocks AS c
JOIN nyc_neighborhoods AS n
ON ST_Intersects(c.geom, n.geom)
WHERE n.name = 'Battery Park'
"""
con.sql(query).show()

┌────────────┐
│ population │
│   int128   │
├────────────┤
│      17153 │
└────────────┘



4. **What neighborhood has the highest population density (persons/km2)?**


In [42]:
# for ST_Area, we have both name and geometry, so we need to group by both
query = """
SELECT n.name, SUM(c.POPN_TOTAL) / (ST_Area(n.geom) * 1000000) AS population_density
FROM nyc_census_blocks AS c
JOIN nyc_neighborhoods AS n
ON ST_Intersects(c.geom, n.geom)
GROUP BY n.name, n.geom
ORDER BY population_density DESC
LIMIT 1
"""
con.sql(query).show()

┌───────────────────┬───────────────────────┐
│       NAME        │  population_density   │
│      varchar      │        double         │
├───────────────────┼───────────────────────┤
│ North Sutton Area │ 6.843513283772679e-08 │
└───────────────────┴───────────────────────┘



When you're finished, you can check your answers [here](https://postgis.net/workshops/postgis-intro/joins_exercises.html).

# Ship-to-Ship Transfer Detection

Now for a less structured exercise. We're going to look at ship-to-ship transfers. The idea is that two ships meet up in the middle of the ocean, and one ship transfers cargo to the other. This is a common way to avoid sanctions, and is often used to transfer oil from sanctioned countries to other countries. We're going to look at a few different ways to detect these transfers using AIS data.

In [ ]:
%pip install duckdb duckdb-engine jupysql

In [ ]:
import duckdb
import pandas as pd

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:///:memory:

In [ ]:
%%sql
INSTALL httpfs;
LOAD httpfs;
INSTALL spatial;
LOAD spatial;

## Step 1

Create a spatial database using the following AIS data:

https://storage.googleapis.com/qm2/casa0025_ships.csv

Each row in this dataset is an AIS 'ping' indicating the position of a ship at a particular date/time, alongside vessel-level characteristics.

It contains the following columns:
* `vesselid`: A unique numerical identifier for each ship, like a license plate
* `vessel_name`: The ship's name
* `vsl_descr`: The ship's type
* `dwt`: The ship's Deadweight Tonnage (how many tons it can carry)
* `v_length`: The ship's length in meters
* `draught`: How many meters deep the ship is draughting (how low it sits in the water). Effectively indicates how much cargo the ship is carrying
* `sog`: Speed over Ground (in knots)
* `date`: A timestamp for the AIS signal
* `lat`: The latitude of the AIS signal (EPSG:4326)
* `lon`: The longitude of the AIS signal (EPSG:4326)

Create a table called 'ais' where each row is a different AIS ping, with no superfluous information. Construct a geometry column.

Create a second table called 'vinfo' which contains vessel-level information with no superfluous information.

You can set a spatial index on each of these tables as follows:

`CREATE INDEX index_name ON table_name USING RTREE(geom);`

## Step 2

Use a spatial join to identify ship-to-ship transfers in this dataset.
Two ships are considered to be conducting a ship to ship transfer IF:

* They are within 500 meters of each other
* For more than two hours
* And their speed is lower than 1 knot

Some things to consider: make sure you're not joining ships with themselves. Try working with subsets of the data first while you try different things out.